In [1]:


#频繁项集：经常一起出现的物品集合
#关联规则：两种物品存在的某种很强的关系

#支持度support:数据集中包含该项集的记录所占总交易记录的比例
#最小支持度：min_support,最小的比例，表示一种下界

#可信度：confidence,针对某种关联规则{A}——>{B}的可信度=support({A,B}) / support({A})
#最小可信度：min_confidence,最小的可信度

#Apriori作用：减少可能感兴趣的项集，避免物品很多时，需要计算的项集呈指数增长，N种物品，(2**N-1)种的项集组合
#Apriori原理：如果某个项集是频繁的，那么他的所有子集也是频繁的
#延伸：如果某个项集是非频繁的，那么他的所有超集也是非频繁的




In [2]:
#任务1：发现频繁项集——————利用Apriori,只保留k个元素的频繁项集

#交易集合
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
#创建C1，单元素集合，[[1],[2],[3],[4],[5]]
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)#use frozen set so we
    #can use it as a key in a dict 
    #对C1中的[i]都做frozenset()操作，改成不可变的集合
dataSet = loadDataSet()
print dataSet
C1 = createC1(dataSet)
print C1
#交易记录
D = map(set,dataSet)
print D





[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
[frozenset([1]), frozenset([2]), frozenset([3]), frozenset([4]), frozenset([5])]
[set([1, 3, 4]), set([2, 3, 5]), set([1, 2, 3, 5]), set([2, 5])]


In [3]:
#去除交易集合D中，不满足最小支持度minSupport的包含k个元素的项集
#得到满足条件的，频繁项集Lk（用list存储）
def scanD(D, Ck, minSupport):
    ssCnt = {} #字典：{[i]:出现次数}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):#是子集合，[i]出现在某条交易记录中
                if not ssCnt.has_key(can): 
                    ssCnt[can]=1
                else: 
                    ssCnt[can] += 1
    numItems = float(len(D)) #float()是为了除法有小数
    Lk = [] #返回的满足条件的频繁项集的集合
    supportData_k = {}#记录{项集:支持度}
    for key in ssCnt: #遍历字典的key
        support = ssCnt[key]/numItems
        if support >= minSupport:
            Lk.insert(0,key) #在列表首添加
        supportData_k[key] = support
    return Lk, supportData_k

#最小支持度=0.5
L1,supportData_1 = scanD(D,C1,0.5)
print L1
print supportData_1

[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])]
{frozenset([4]): 0.25, frozenset([5]): 0.75, frozenset([2]): 0.75, frozenset([3]): 0.75, frozenset([1]): 0.5}


In [4]:
#频繁项集链表Lk,项集元素的个数k
#创建k个元素的候选项集Ck
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]  #k个元素，求前面(k-2)个元素都相同的集合的并集,k=(k-2)+1+1
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union并集
    return retList


def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet) #初始的候选项集C1
    D = map(set, dataSet) #交易集合
    L1, supportData = scanD(D, C1, minSupport) #初始的频繁项集L1
    
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK) #添加字典
        L.append(Lk)
        k += 1
    return L, supportData

L,supportData = apriori(dataSet)
print L
print supportData

[[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])], [frozenset([1, 3]), frozenset([2, 5]), frozenset([2, 3]), frozenset([3, 5])], [frozenset([2, 3, 5])], []]
{frozenset([5]): 0.75, frozenset([3]): 0.75, frozenset([2, 3, 5]): 0.5, frozenset([1, 2]): 0.25, frozenset([1, 5]): 0.25, frozenset([3, 5]): 0.5, frozenset([4]): 0.25, frozenset([2, 3]): 0.5, frozenset([2, 5]): 0.75, frozenset([1]): 0.5, frozenset([1, 3]): 0.5, frozenset([2]): 0.75}


In [8]:
#任务2：从频繁项集中挖掘关联规则：使用可信度confidence

#特点：某条规则不满足最小可信度要求，则该规则左边的所有子集也不会满足最小可信度要求
#{0,1,2}——>{3}不满足。则{0,1,2}子集作为左边（前件）的规则也不满足

#或者：{0,1,2}——>{3}不满足。则所有包含{3}作为右边（后件）的规则也不满足。(代码采用这个原理)

#步骤：首先从一个频繁项集开始，接着创建一个规则列表，其中规则右部（后件）从只有一个元素开始
#然后对这些规则进行测试 ，接下来合并符合条件的后件（从右部有1个元素到2个元素，2->3,3->4，
# 到频繁项集的大小-1,如此循环，测试规则是否符合,）   P210图11-4
            
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        #因为无法从单元素项集中构建关联规则,所以没有下标0
        print "-------------this is in L(%d)------------"%(i)
        print "--------",L[i],'-----------'
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet] #单元素集合
            #frozenset([1,2])——>[frozenset([1]),frozenset([2])]
            if (i > 1): #频繁项集的元素数目>2
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else: #频繁项集中元素=2,直接计算可信度
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

#返回一个满足最小可信度的规则列表:yusna
def calcConf(freqSet, H, supportData, bigRuleList, minConf=0.7):
    print 'the frequent set: ',freqSet,'-----------'
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print freqSet-conseq,'-->',conseq,'conf:',conf
            bigRuleList.append((freqSet-conseq, conseq, conf)) #元组（前件，后件，可信度）
            prunedH.append(conseq)  #保存的是后件
    print "stored the right set:",prunedH
    return prunedH

def rulesFromConseq(freqSet, H, supportData, bigRuleList, minConf=0.7):
    m = len(H[0]) #Hm的元素个数
   
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates,H(m+1)个元素集合
        Hmp1 = calcConf(freqSet, Hmp1, supportData, bigRuleList, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge,接着合并
            rulesFromConseq(freqSet, Hmp1, supportData, bigRuleList, minConf)
            
            
#测试一些函数
for freqSet in L[2]:
    print freqSet
    H1 = [frozenset([item]) for item in freqSet] #单元素集合
    print H1
    
            

frozenset([2, 3, 5])
[frozenset([2]), frozenset([3]), frozenset([5])]


In [9]:
rules = generateRules(L,supportData,minConf=0.7)
print rules

-------------this is in L(1)------------
-------- [frozenset([1, 3]), frozenset([2, 5]), frozenset([2, 3]), frozenset([3, 5])] -----------
the frequent set:  frozenset([1, 3]) -----------
frozenset([1]) --> frozenset([3]) conf: 1.0
stored the right set: [frozenset([3])]
the frequent set:  frozenset([2, 5]) -----------
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0
stored the right set: [frozenset([2]), frozenset([5])]
the frequent set:  frozenset([2, 3]) -----------
stored the right set: []
the frequent set:  frozenset([3, 5]) -----------
stored the right set: []
-------------this is in L(2)------------
-------- [frozenset([2, 3, 5])] -----------
this is Hmp1 [frozenset([2, 3]), frozenset([2, 5]), frozenset([3, 5])] -----------
the frequent set:  frozenset([2, 3, 5]) -----------
stored the right set: []
-------------this is in L(3)------------
-------- [] -----------
[(frozenset([1]), frozenset([3]), 1.0), (frozenset([5]), frozenset([2]), 1.0), 

In [10]:
#减小可信度=0.5        
rules = generateRules(L,supportData,minConf=0.5)
print rules


-------------this is in L(1)------------
-------- [frozenset([1, 3]), frozenset([2, 5]), frozenset([2, 3]), frozenset([3, 5])] -----------
the frequent set:  frozenset([1, 3]) -----------
frozenset([3]) --> frozenset([1]) conf: 0.666666666667
frozenset([1]) --> frozenset([3]) conf: 1.0
stored the right set: [frozenset([1]), frozenset([3])]
the frequent set:  frozenset([2, 5]) -----------
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0
stored the right set: [frozenset([2]), frozenset([5])]
the frequent set:  frozenset([2, 3]) -----------
frozenset([3]) --> frozenset([2]) conf: 0.666666666667
frozenset([2]) --> frozenset([3]) conf: 0.666666666667
stored the right set: [frozenset([2]), frozenset([3])]
the frequent set:  frozenset([3, 5]) -----------
frozenset([5]) --> frozenset([3]) conf: 0.666666666667
frozenset([3]) --> frozenset([5]) conf: 0.666666666667
stored the right set: [frozenset([3]), frozenset([5])]
-------------this is in L(2)----------

In [11]:
#利用Apriori算法，发现毒蘑菇的相似特征：就是毒蘑菇特征的频繁项集

mushDatSet = [line.split() for line in open('apriori/mushroom.dat').readlines()]
L,supportData = apriori(mushDatSet,0.3)
for item in L[1]:
    if item.intersection('2'):#交集
        print item
        


frozenset(['2', '59'])
frozenset(['39', '2'])
frozenset(['2', '67'])
frozenset(['2', '34'])
frozenset(['2', '23'])
frozenset(['2', '86'])
frozenset(['76', '2'])
frozenset(['90', '2'])
frozenset(['2', '53'])
frozenset(['93', '2'])
frozenset(['63', '2'])
frozenset(['2', '28'])
frozenset(['2', '85'])
frozenset(['2', '36'])
